In [113]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import pandas as pd
import time
import csv

In [27]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

### Read in file for play back

In [98]:
path = r'D:\Users\Student\OneDrive\Bath\Dissertation\Data\MediaPipe_Implementation'
test_data = pd.read_csv(os.path.join(path, r'raw_data.csv'))
if 'Unnamed: 0' in test_data.columns:
    test_data = test_data.drop(columns = ['Unnamed: 0'])
else:
    pass
test_data

,0
0,landmark {\n x: 0.21897092\n y: 0.91752213\n...
1,landmark {\n x: 0.2143432\n y: 0.8994516\n ...
2,landmark {\n x: 0.21381351\n y: 0.87002635\n...
3,landmark {\n x: 0.21300381\n y: 0.84361446\n...
4,landmark {\n x: 0.211342\n y: 0.8254896\n z...
5,landmark {\n x: 0.21141207\n y: 0.8238297\n ...
6,landmark {\n x: 0.2095038\n y: 0.83453614\n ...
7,landmark {\n x: 0.21298854\n y: 0.8461989\n ...
8,landmark {\n x: 0.21681494\n y: 0.85825896\n...
9,landmark {\n x: 0.22012703\n y: 0.86667377\n...


### Clean the dataset

In [111]:
#remove all the \n from the string and place into list
for index in test_data.iterrows():
    x_list = list(index)
    wanted = x_list[1][0]
    x_replace = [str(item).replace('\n',' ') for item in wanted]

''.join(x_replace)
# # get the landmarks - index doesn't matter
# len(x_replace[-1])
# x_replace[-1][1300:]

'landmark {   x: 0.2086877   y: 0.8790742   z: -4.1676925e-05 } landmark {   x: 0.25931135   y: 0.87028074   z: -0.08230013 } landmark {   x: 0.2952926   y: 0.87752014   z: -0.16288011 } landmark {   x: 0.3220781   y: 0.91776854   z: -0.22651383 } landmark {   x: 0.31877896   y: 0.9521524   z: -0.29441243 } landmark {   x: 0.26406828   y: 0.8618276   z: -0.17899367 } landmark {   x: 0.2755541   y: 0.9191599   z: -0.29516956 } landmark {   x: 0.28422523   y: 0.9811046   z: -0.3728073 } landmark {   x: 0.28864756   y: 1.0350913   z: -0.41860455 } landmark {   x: 0.21821192   y: 0.90228987   z: -0.1725682 } landmark {   x: 0.22932997   y: 0.9525325   z: -0.3044476 } landmark {   x: 0.2381471   y: 1.0380586   z: -0.40101194 } landmark {   x: 0.2416828   y: 1.0986311   z: -0.45793787 } landmark {   x: 0.17863846   y: 0.94445586   z: -0.16592076 } landmark {   x: 0.17683023   y: 1.0123315   z: -0.28375828 } landmark {   x: 0.1836833   y: 1.0706424   z: -0.34584546 } landmark {   x: 0.1890386

## Attempt at playback

In [29]:
#create empty list to store all the pd dfs
dataframe_list = []
cap = cv2.VideoCapture(0)
start = time.time()

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands = 10) as hands:
    #initialise the frame count
    frame_counter = 0   
    while cap.isOpened():
               
        ret, frame = cap.read()
        
        # count the frame number manually as using cap.get(cv2.CAP_PROP_POS_FRAMES) is tidious
        frame_counter += 1
               
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
                
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
               
        # Rendering results
        # checking if there are any hands in the frame
        if results.multi_hand_landmarks:
        
#             print(results.multi_hand_landmarks)
            
            # looping through each feature in the frame
            for num, hand in enumerate(results.multi_hand_landmarks):
                
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
                        
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
for index in 

### Code for static image

In [ ]:
with mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5) as hands:
    for idx, file in enumerate(file_list):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print handedness and draw hand landmarks on the image.
    print('Handedness:', results.multi_handedness)
    if not results.multi_hand_landmarks:
          continue
    image_height, image_width, _ = image.shape
    annotated_image = image.copy()
    
    for hand_landmarks in results.multi_hand_landmarks:
        print('hand_landmarks:', hand_landmarks)
        print(f'Index finger tip coordinates: (',
              f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
              f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
      )
          mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))